## Loading Structure Objects from Files

Now that we know a `Structure` is a Python *object* created from the
`Structure` *class*, the next step is learning how to create these objects
directly from real structure files.

`Structure.from_file()` is a **class method**.  
It reads a file (CIF, POSCAR/CONTCAR, XYZ, PDB, etc.) and returns
a full **Structure object**.

This is just object creation, but from a file instead of typing numbers.


In [1]:
from pymatgen.core import Structure

s = Structure.from_file("POSCAR") # Load structure
                                  # doesn't have to be POSCAR format
                                  # can be any file supported by pymatgen

print(s.formula)
print(s.lattice)
print(s.volume)

B1 N1
0.000000 1.785000 1.785000
1.785000 0.000000 1.785000
1.785000 1.785000 0.000000
11.374823249999999


Everything here is an **attribute** on the `Structure` object,
exactly like attributes on any Python class we defined earlier.

So even though a `Structure` is scientifically complex, the underlying idea
is still simple:

- `Structure` = *class*  
- `s` = *object instance*  

# Loading Data from a JSON File

We just learned how to load a `Structure` object from a file like a CIF or POSCAR.

Now we want to load **calculation results** stored in a JSON file.
These JSONs often contain:
- energies  
- metadata  
- full structures  
- and other information from earlier DFT calculations  

Before we load real pymatgen objects, we start by loading the file
as plain JSON data.

In [3]:
import json

with open("computed_structure_entries.json", "r") as f:
    raw_data = json.load(f)

raw_data[0]   # look at the first dictionary

{'@module': 'pymatgen.entries.computed_entries',
 '@class': 'ComputedStructureEntry',
 'energy': -151.14305081,
 'composition': {'Li': 4.0, 'Nb': 4.0, 'Cl': 16.0, 'O': 4.0},
 'entry_id': 'relax_1',
 'correction': 0.0,
 'energy_adjustments': [],
 'parameters': {'potcar_spec': [{'titel': 'PAW_PBE Li_sv 10Sep2004',
    'hash': None,
    'summary_stats': {}},
   {'titel': 'PAW_PBE Nb_sv 25May2007', 'hash': None, 'summary_stats': {}},
   {'titel': 'PAW_PBE Cl 06Sep2000', 'hash': None, 'summary_stats': {}},
   {'titel': 'PAW_PBE O 08Apr2002', 'hash': None, 'summary_stats': {}}],
  'potcar_symbols': ['PAW_PBE Li_sv 10Sep2004',
   'PAW_PBE Nb_sv 25May2007',
   'PAW_PBE Cl 06Sep2000',
   'PAW_PBE O 08Apr2002'],
  'hubbards': {},
  'is_hubbard': False,
  'run_type': 'PBE+vdW-DFT-D3-BJ',
  'INCAR': {'PREC': 'Accurate',
   'ALGO': 'Normal',
   'ADDGRID': True,
   'ISPIN': 1,
   'NELM': 200,
   'NELMIN': 5,
   'IBRION': 2,
   'EDIFF': 1e-08,
   'EDIFFG': -0.0005,
   'NSW': 1000,
   'ISIF': 3,
   'E

### Why do we write with open(...) instead of just open(...)?

When Python opens a file, that file uses some of your computer’s resources.
If you open a file like this:

In [5]:
f = open("computed_structure_entries.json", "r")
data = json.load(f)

then you also need to remember to close it:

In [6]:
f.close()

If you forget to close a file, it can stay open in the background.
This can cause problems, especially when your program loads many files or runs for a long time.

The with statement solves this.

### What with does

When you write:

In [8]:
with open("computed_structure_entries.json", "r") as f:
    data = json.load(f)

### Python automatically:

opens the file

lets you work with it inside the with block

closes the file for you when the block ends

even if an error happens inside the block

This means you never have to remember to call f.close() yourself.

### A simple analogy

Using with is like borrowing a library book where the library automatically takes it back as soon as you leave the building.
You don’t need to remember to return it.

### summary

The with statement is the safest and cleanest way to open files because it automatically closes them, even if something goes wrong.

### Loading JSON Data as Plain Python Dictionaries

Now that we understand why the `with` statement is used when working with files,
we can safely load our JSON file and look inside it.

Remember: JSON files store *data*, not Python objects.

Let’s load the file using `with open(...)` and inspect the first entry.


In [3]:
from monty.serialization import loadfn

entries = loadfn("computed_structure_entries.json")

entries[0]

relax_1 ComputedStructureEntry - Li4 Nb4 Cl16 O4 (LiNbCl4O)
Energy (Uncorrected)     = -151.1431 eV (-5.3980  eV/atom)
Correction               = 0.0000    eV (0.0000   eV/atom)
Energy (Final)           = -151.1431 eV (-5.3980  eV/atom)
Energy Adjustments:
  None
Parameters:
  potcar_spec            = [{'titel': 'PAW_PBE Li_sv 10Sep2004', 'hash': None, 'summary_stats': {}}, {'titel': 'PAW_PBE Nb_sv 25May2007', 'hash': None, 'summary_stats': {}}, {'titel': 'PAW_PBE Cl 06Sep2000', 'hash': None, 'summary_stats': {}}, {'titel': 'PAW_PBE O 08Apr2002', 'hash': None, 'summary_stats': {}}]
  potcar_symbols         = ['PAW_PBE Li_sv 10Sep2004', 'PAW_PBE Nb_sv 25May2007', 'PAW_PBE Cl 06Sep2000', 'PAW_PBE O 08Apr2002']
  hubbards               = {}
  is_hubbard             = False
  run_type               = PBE+vdW-DFT-D3-BJ
  INCAR                  = ADDGRID = True
ALGO = Normal
EDIFF = 1e-08
EDIFFG = -0.0005
ENCUT = 780.0
GGA = Pe
IBRION = 2
ISIF = 3
ISMEAR = 0
ISPIN = 1
IVDW = 12
KSPACING = 0.

Now we have a real:

``pymatgen.entries.computed_entries.ComputedStructureEntry``

This means we can use attributes like:

- `.energy`
- `.structure`
- `.composition`
- `.structure.volume`

Exactly like any other pymatgen object.

In [4]:
e = entries[0]
print("Energy:", e.energy)
print("Volume:", e.structure.volume)
print("Composition:", e.composition)

Energy: -151.14305081
Volume: 644.6144125437787
Composition: Li4 Nb4 Cl16 O4
